<a href="https://colab.research.google.com/github/blackwithwhitegreen/Pytorch/blob/main/Pytorch_pipline(basic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

In [3]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
df.shape

(569, 33)

In [6]:
df.drop(columns={'id','Unnamed: 32'},inplace=True)

In [7]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [26]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2,stratify=df.iloc[:, 0])

#Scaling Data because we need to send the same scale of data to the nural network.

In [27]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
X_train

array([[-1.1037049 , -1.02021574, -1.11241567, ..., -1.38414778,
         1.06340309, -0.22368131],
       [-0.06742619, -0.48624898, -0.13418633, ..., -0.99466011,
        -0.89247727, -0.68836934],
       [ 0.34089014,  1.36086352,  0.43743702, ...,  1.34010641,
         1.83395109,  0.80550841],
       ...,
       [ 1.28424168, -0.39687257,  1.30961304, ...,  1.88616042,
         1.46015334,  0.42233487],
       [ 1.53204746, -0.0989512 ,  1.54710698, ...,  1.27223529,
         0.09775889,  0.83489799],
       [ 0.58587995, -0.7475031 ,  0.59713122, ...,  0.14773424,
         0.4010597 ,  0.29756779]])

In [29]:
y_train

,diagnosis
60,B
125,B
28,M
181,M
165,B
...,...
138,M
381,B
45,M
300,M


In [30]:
y_test

,diagnosis
563,M
363,B
533,M
16,M
177,M
...,...
128,B
38,M
519,B
280,M


#For y there are two classes preset, and out model will not understand it so, we need to label it by using ***`Label Encoder`***.

In [31]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

#Converting Numpy arrays to Pytorch tensors.

In [33]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [35]:
X_train_tensor.shape

torch.Size([455, 30])

#Defining Model

In [53]:
class MySimpleNN():

  def __init__(self,X):

    self.weights = torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
    self.bias = torch.zeros(1,dtype=torch.float64,requires_grad=True)


  def forward(self,X):
    z = torch.matmul(X,self.weights)+ self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self,y_pred,y):
    #clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred,epsilon,1-epsilon)

    #clauclate loss
    loss = -(y * torch.log(y_pred)+ (1-y) * torch.log(1-y_pred)).mean()
    return loss

In [59]:
learning_rate =0.1
epoches = 25

#Training Pipeline

In [60]:
#Creating Model
model = MySimpleNN(X_train_tensor)

#define loop
for epoch in range(epoches):


  #forward pass
  y_pred = model.forward(X_train_tensor)
  # print(y_pred)

  #loss calculate
  loss = model.loss_function(y_pred,y_train_tensor)
  print(f'Epoch: {epoch+1},Loss: {loss.item()}')


  #backward pass
  loss.backward()

  #parameters
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  # Zero Gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  #print loss in each epoch
  print(f'Epoch: {epoch + 1},Loss: {loss.item()}')

Epoch: 1,Loss: 3.6795489128736185
Epoch: 1,Loss: 3.6795489128736185
Epoch: 2,Loss: 3.544456507996272
Epoch: 2,Loss: 3.544456507996272
Epoch: 3,Loss: 3.4067152078622525
Epoch: 3,Loss: 3.4067152078622525
Epoch: 4,Loss: 3.2655413546743626
Epoch: 4,Loss: 3.2655413546743626
Epoch: 5,Loss: 3.1228705704082826
Epoch: 5,Loss: 3.1228705704082826
Epoch: 6,Loss: 2.976067836192733
Epoch: 6,Loss: 2.976067836192733
Epoch: 7,Loss: 2.8238871995186687
Epoch: 7,Loss: 2.8238871995186687
Epoch: 8,Loss: 2.6685635900480253
Epoch: 8,Loss: 2.6685635900480253
Epoch: 9,Loss: 2.5044355976125376
Epoch: 9,Loss: 2.5044355976125376
Epoch: 10,Loss: 2.340986146315047
Epoch: 10,Loss: 2.340986146315047
Epoch: 11,Loss: 2.180743624067122
Epoch: 11,Loss: 2.180743624067122
Epoch: 12,Loss: 2.0229841265594417
Epoch: 12,Loss: 2.0229841265594417
Epoch: 13,Loss: 1.865045105018131
Epoch: 13,Loss: 1.865045105018131
Epoch: 14,Loss: 1.7111960724780635
Epoch: 14,Loss: 1.7111960724780635
Epoch: 15,Loss: 1.5633547609655503
Epoch: 15,Los

In [63]:
model.bias

tensor([-0.1287], dtype=torch.float64, requires_grad=True)

#here we need to thrshold where the values are being check if it is more than this then goes to 0 otherwise 1.

In [73]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9999).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.6315789222717285
